In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

Mounted at /content/gdrive/


In [2]:
%cd '/content/gdrive/My Drive/IDLSProject-main'

/content/gdrive/My Drive/IDLSProject-main


In [3]:
!pwd

/content/gdrive/My Drive/IDLSProject-main


In [1]:
import numpy as np

In [9]:
import pandas as pd
import os
import pickle
import torch
import torch.utils.data
from sklearn.model_selection import train_test_split

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_amazon.pkl'), "rb") as f:
    word_dict = pickle.load(f)
    
train = pd.read_csv(os.path.join(data_dir, 'train_amazon.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_amazon.csv'), header=None, names=None)
print(train.shape, test_sample.shape)


test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

#drop rows
test.drop(test.tail(227).index,inplace = True)
val.drop(val.tail(228).index,inplace = True)
test.shape, val.shape

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape, val_y.shape)

(83000, 502) (21975, 502)
torch.Size([10760, 1]) torch.Size([10760, 1])


In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 10
batch_size = 50
learning_rate = 0.001
seq_len = 500
dropout = 0.5
filter_size = 100
vocab_size = 10000
embed_dims = 32
hidden_size = 100
kernel_size = [3,4,5]

In [22]:
# write to file.
filename = "cnn-lstm-amazon.csv"
def write_to_csv(epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [23]:
import torch.nn as nn
import numpy as np
class Combine(nn.Module):
    def __init__(self,vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len,hidden_size):
        super(Combine, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size,embed_size)
        self.conv1 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[0],embed_size])
        self.conv2 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[1],embed_size])
        self.conv3 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[2],embed_size])
        self.mp1 = torch.nn.MaxPool1d(seq_len+1-kernel_size[0])
        self.mp2 = torch.nn.MaxPool1d(seq_len+1-kernel_size[1])
        self.mp3 = torch.nn.MaxPool1d(seq_len+1-kernel_size[2])
        self.dropout = torch.nn.Dropout(dropout)
        #self.cnn = CNN(vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len)
        self.lstm = nn.LSTM(
            input_size=3*filter_size, 
            hidden_size=hidden_size, 
            num_layers=1,
            batch_first=True)
        self.dense = nn.Linear(hidden_size,1)
        self.sig = nn.Sigmoid()
        self.word_dict = None


    def forward(self, x):
        batch_size = x.size(0)
        #print(batch_size)
        embed_input = self.embedding(x)
        embed_input.unsqueeze_(1)
        x1 = torch.tanh(self.dropout(self.conv1(embed_input))).squeeze(3)
        x2 = torch.tanh(self.dropout(self.conv2(embed_input))).squeeze(3)
        x3 = torch.tanh(self.dropout(self.conv3(embed_input))).squeeze(3)
        f1 = self.mp1(x1).squeeze(2)
        f2 = self.mp2(x2).squeeze(2)
        f3 = self.mp3(x3).squeeze(2)
        c_out = torch.cat([f1,f2,f3],dim=1)
        r_in = c_out.view(batch_size, 1, -1)
        lstm_out, _ = self.lstm(r_in)
        out = self.dense(lstm_out[:, -1, :])
        return self.sig(out)

In [24]:
import time
import numpy as np 
def train(model, train_dl, val_dl, epochs, optimizer, loss_fn, device):
    train_loss_epoch = []
    train_acc_epoch = []
    val_loss_epoch = []
    val_accuracy_epoch = []
    time_train = []
    for epoch in range(epochs):
        start = time.time()
        model.train()
        total_loss = 0
        train_acc = 0
        total = 0
        correct = 0
        for batch in train_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            prediction = model(batch_X)
            loss = loss_fn(prediction, batch_y)
            loss.backward()
            optimizer.step()
            result = np.round(prediction.detach().cpu())
            total_loss += loss.data.item()
            total += batch_y.size(0)
            correct += (result == batch_y.cpu()).sum().item()
            train_acc = correct/total
        train_loss_epoch.append(np.round(total_loss / len(train_dl), 3))
        train_acc_epoch.append(np.round(train_acc*100,3))
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
        
        with torch.no_grad():
            model.eval()
            correct = 0
            total = 0
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                prediction = model(inputs_val)
                loss = loss_fn(prediction, labels_val)
                val_loss.append(loss.item())
                result = np.round(prediction.cpu())
                total += labels_val.size(0)
                correct += (result == labels_val.cpu()).sum().item()
            val_accuracy_epoch.append(np.round((correct/total)*100, 3))
            val_loss_epoch.append(np.round(np.mean(val_loss),3))
            end = time.time() - start
            print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(correct/total))
            time_train.append(np.round(end,3))
    write_to_csv(epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_accuracy_epoch, time_train)
    return model


def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    acc = correct/total
    append_to_csv(epochs, np.round(acc*100, 3))
    print("Accuracy:", (correct/total)*100)

In [ ]:
import torch.optim as optim
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Combine(vocab_size, embed_dims, filter_size, kernel_size, dropout, seq_len,hidden_size).to(device).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()
train(model, train_dl, val_dl, epochs, optimizer, loss_fn, device)

Epoch: 0, BCELoss: 0.39424134833985064
Val Loss: 0.317 	Val Acc: 0.865


In [ ]:
torch.save(model.state_dict(),"./cnn-lstm-amazon.pth")

In [ ]:
# def test(model, test_dl, epochs):
#     model.eval()
#     correct = 0
#     total = 0
# #     results = []
# #     labels = []
#     with torch.no_grad():
#         for batch in test_dl:         
#             batch_X, batch_y = batch
#             batch_X = batch_X.to(device)
#             prediction = model(batch_X)
#             result = np.round(prediction.cpu())
# #             results.extend(list(result.numpy()))
# #             labels.extend(list(batch_y.numpy()))
#             total += batch_y.size(0)
#             correct += (result == batch_y).sum().item()
#     acc = correct/total
#     print("Accuracy:", correct/total)

In [ ]:
test(model, test_dl, epochs)